In [1]:
import pandas as pd
import numpy as np
import time
import re
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

In [2]:
# import sys
# sys.path.insert(0, 'usr/lib/chromium-browser/chromedriver')

In [3]:
options = webdriver.ChromeOptions()
# option.add_argument('headless')
options.add_argument('disable-gpu')
options.add_argument('lang=ko_KR')
driver = webdriver.Chrome('./chromedriver', options=options)

In [4]:
# 댓글 버튼 유무
def is_comment(url_id):
    try:
        driver.find_element_by_xpath(f'//*[@id="Comi{url_id}"]')
        return 0
    except:
        return 1

In [5]:
# 해시태그 갯수
def tag_counts():
    hash_tags = []
    blog_tags = []
    try:
        tags = []
        for i in range(len(driver.find_elements_by_class_name('tag'))):
            tag = driver.find_elements_by_class_name('tag')[i].text
            tags.append(tag.replace('#',''))
            join_tags = ', '.join(tags)
        
        if tags == []:
            hash_tags = np.NaN
        else:
            hash_tags = join_tags
            
    except Exception as e:
        print(e,'!!error1!!')
        return 0

    try:
        blogtags = []
        for j in range(len(driver.find_elements_by_class_name('ell'))):
            blogtag = driver.find_elements_by_class_name('ell')[j].text
            blogtags.append(blogtag.replace('#',''))
            join_blogtags = ','.join(blogtags)
        
        if blogtags == []:
            blog_tags = np.NaN
        else:
            blog_tags = join_blogtags
    
    except Exception as e:
        print(e,'error2')
        return 0
    
    return hash_tags, blog_tags

In [6]:
def writeDate():
    pass

In [7]:
lst_keyword = ['블루투스 마우스','블루투스 키보드']


Data = pd.DataFrame()


for keyword in lst_keyword:
    for page in range(2):
        print(f'======================================{page}=====================================')
        try:
            url = f'https://search.naver.com/search.naver?date_from=&date_option=0&date_to=&dup_remove=1&nso=&post_blogurl=&post_blogurl_without=\
            &query={keyword}&sm=tab_pge&srchby=all&st=sim&where=post&start={page*10 + 1}'
            driver.get(url)
            time.sleep(2.5)
            
            lst_title = []
            lst_content = []
            lst_comment = []
            lst_image = []
            lst_video = []
            lst_link = []
            lst_date = []
            lst_category = []
            lst_hashTag = []
            lst_blogTag = []
            
            df = pd.DataFrame()
            
            for i in range(1, 5):
                try:
                    title = driver.find_element_by_xpath(f'//*[@id="sp_blog_{i}"]/dl/dt/a').text
                    lst_title.append(title)
                    print('title: ', title)
                    
                    # 타이틀 클릭
                    driver.find_element_by_xpath(f'//*[@id="sp_blog_{i}"]/dl/dt/a').click()
                    time.sleep(2.5)
                    
                    #window 탭 이동
                    driver.switch_to.window(driver.window_handles[1])
                    time.sleep(2.5)
                    
                    # 프레임 링크 전환
                    driver.switch_to.frame("mainFrame")
                    
                    # 블로그 content
                    current_url = driver.current_url
                    url_id = current_url.split('/')[-1]
                    # content = driver.find_element_by_xpath('//*[@id="post-view{0}"]/div'.format(current_url)).text
                    content = driver.find_element_by_class_name("se-main-container").text
                    # print(content)
                    # print(current_url)
                    
                    # 댓글 버튼의 유무
                    comment = is_comment(url_id)
                    # print(comment)
                    
                    # 태그 갯수 -> list로 반환
                    hashTags, blogTags = tag_counts()
                    
                    #블로그 날짜
                    date=driver.find_element_by_class_name("blog2_container").text
                    date=re.sub('[^0-9|.]', '', date)
                    date=date[:-5]
                    print(date)
                    
                    try: 
                        # 이미지 갯수
                        image = len(driver.find_elements_by_class_name('se-image-resource'))  
                        # print(image)
                        
                        # 비디오 갯수
                        video = len(driver.find_elements_by_class_name('u_rmcplayer_control_bg'))  
                        # print(video)
                        
                        # 링크 갯수
                        link1 = len(driver.find_elements_by_class_name("se-link"))  
                        link2 = len(driver.find_elements_by_class_name("se-oglink-url"))
                        links = (link1 + link2)
                        # print(links)
                        
                        # 카테고리 갯수
                        category = driver.find_element_by_class_name('bcc').text 
                        category = re.sub('[^0-9]', '', category)
                        
                        
                        lst_image.append(image)
                        lst_video.append(video)
                        lst_link.append(links)
                        lst_category.append(category)
                        
                    except:
                        lst_image.append(0)
                        lst_video.append(0)
                        lst_link.append(0)
                        lst_category.append(0)

                    lst_content.append(content)
                    lst_comment.append(comment)
                    lst_hashTag.append(hashTags)
                    lst_blogTag.append(blogTags)
                    lst_date.append(date)
                     
                except:
                    continue
                
                finally:
                    print('.')
                    driver.close()
                    time.sleep(2.0)
                    driver.switch_to.window(driver.window_handles[0])

                    df = pd.DataFrame({'title':lst_title, 'content':lst_content, 'comment': lst_comment,
                                       'image_cnt':lst_image, 'video_cnt':lst_video, 'link_cnt':lst_video,
                                       'hash_tag':lst_hashTag, 'blog_tag':lst_blogTag, 'categort_cnt':lst_category,
                                       'date': lst_date})
                    df.to_csv(f'./crawling/IT_crawling_{keyword}_page{page+1}.csv')
                    # print(df.head())
        
        except Exception as e:
            print(e)
            continue
        
        finally:
            print('> 끝!')
            Data = pd.concat([Data, df], ignore_index=True)
            Data.to_csv(f'./crawling/IT_crawling_{keyword}.csv')
        
driver.close()

Data.head()

======================================0=====================================
title:  로지텍 블루투스 마우스 MX Anywhere 3 for Mac 맥북에 추천
2020.10.5
.
title:  블루투스 마우스 로지텍 페블 M350 솔직 후기 / 무소음 마우스...
2020.5.16
.
title:  블루투스 무선마우스 아이리버 괜춘다~!!
2020.8.30
.
title:  추천하는 블루투스 마우스 로지텍 MX Master3

.
> 끝!
======================================1=====================================
title:  [필립스] 멀티 페어링 무선마우스 : 블루투스마우스 추천!
2020.10.11
.
title:  맥북에 딱좋은 블루투스 마우스 매직마우스 2 구입기
2020.5.15
.
title:  로지텍 MX Anywhere 3 휴대용 무선 블루투스 마우스 추천 리뷰
2020.10.
.
title:  손목보호마우스 끝판왕 블루투스마우스 트랙포인트
2020.9.25
.
> 끝!
======================================0=====================================
title:  풀 배열 블루투스 키보드 로지텍 K780, 멀티 페어링은 기본!
2020.8.1
.
title:  엑토 접이식 블루투스 키보드 터치패드도 돼요
2020.6.3
.
title:  [아이노트 블루투스 키보드]... 잘 어울리는 가성비 키보드
2020.10.7
.
title:  블루투스 키보드 로지텍 K580 K380, 취향에 맞게 선택하자
2020.6.22
.
> 끝!
======================================1=====================================
title:  아이패드 7세대 키보드 블루투스 방식의 로지텍 슬림 폴리오...
2020.4.8
.


,title,content,comment,image_cnt,video_cnt,link_cnt,hash_tag,blog_tag,categort_cnt,date
0,로지텍 블루투스 마우스 MX Anywhere 3 for Mac 맥북에 추천,아이맥 27인치 5K를 시작으로 맥 OS를 사용하기 시작하면서 가장 먼저 익숙해지기...,0,18,1,1,NaN,"블루투스마우스,로지텍블루투스마우스,MXAnywhere3forMac",24112019128264201617932015711,2020.10.5
1,블루투스 마우스 로지텍 페블 M350 솔직 후기 / 무소음 마우스...,Carl's Review\n아이패드에도 마우스가 블루투스로 연결이 된다! \nLOG...,0,17,0,0,NaN,"아이패드,아이패드마우스,아이패드블루투스마우스,아이패드로지텍,아이패드미니마우스,로지텍...",8,2020.5.16
2,블루투스 무선마우스 아이리버 괜춘다~!!,매일 1일1포스팅을 하면서도 매일 쓰는 주제가 다르니 ㅎㅎ 오늘은 오랜만에 \n노트...,0,25,2,2,NaN,"블루투스마우스,블루투스무선마우스,무선마우스",1420182214020175268242016112162016919212016102,2020.8.30
3,추천하는 블루투스 마우스 로지텍 MX Master3,로지텍 마우스는 저가 보급형에서부터 고가 프리미엄 라인까지 꽤 다양하게 출시가 되...,0,20,1,1,NaN,"로지텍,블루투스마우스,MXMaster3,로지텍MXMaster3,로지텍마우스,마우스추...",803,
4,[필립스] 멀티 페어링 무선마우스 : 블루투스마우스 추천!,오늘은 집에사 사용하기 편리한 \n멀티 페어링 무선마우스 소개!\n필립스_SPK76...,0,24,1,1,NaN,"필립스,블루투스마우스,무선마우스,마우스,마우스추천,블루투스마우스추천,무선마우스추천,...",8201802141502018214102014327,2020.10.11


**전처리 시 고려해야할 사항**

"""
네이버 동영상 플레이어
키보드와 마우스 터치펜으로 아이패드 7세대 활용하기
3
1,708
01
:
45
재생시간, 이 동영상의 길이는 1분 45초 입니다.
화질 선택 옵션
270p 360p 480p 720p
화질 선택 옵션
접기/펴기 """ ---> 처리

In [8]:
Data.head()

,title,content,comment,image_cnt,video_cnt,link_cnt,hash_tag,blog_tag,categort_cnt,date
0,로지텍 블루투스 마우스 MX Anywhere 3 for Mac 맥북에 추천,아이맥 27인치 5K를 시작으로 맥 OS를 사용하기 시작하면서 가장 먼저 익숙해지기...,0,18,1,1,NaN,"블루투스마우스,로지텍블루투스마우스,MXAnywhere3forMac",24112019128264201617932015711,2020.10.5
1,블루투스 마우스 로지텍 페블 M350 솔직 후기 / 무소음 마우스...,Carl's Review\n아이패드에도 마우스가 블루투스로 연결이 된다! \nLOG...,0,17,0,0,NaN,"아이패드,아이패드마우스,아이패드블루투스마우스,아이패드로지텍,아이패드미니마우스,로지텍...",8,2020.5.16
2,블루투스 무선마우스 아이리버 괜춘다~!!,매일 1일1포스팅을 하면서도 매일 쓰는 주제가 다르니 ㅎㅎ 오늘은 오랜만에 \n노트...,0,25,2,2,NaN,"블루투스마우스,블루투스무선마우스,무선마우스",1420182214020175268242016112162016919212016102,2020.8.30
3,추천하는 블루투스 마우스 로지텍 MX Master3,로지텍 마우스는 저가 보급형에서부터 고가 프리미엄 라인까지 꽤 다양하게 출시가 되...,0,20,1,1,NaN,"로지텍,블루투스마우스,MXMaster3,로지텍MXMaster3,로지텍마우스,마우스추...",803,
4,[필립스] 멀티 페어링 무선마우스 : 블루투스마우스 추천!,오늘은 집에사 사용하기 편리한 \n멀티 페어링 무선마우스 소개!\n필립스_SPK76...,0,24,1,1,NaN,"필립스,블루투스마우스,무선마우스,마우스,마우스추천,블루투스마우스추천,무선마우스추천,...",8201802141502018214102014327,2020.10.11


In [9]:
Data.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         16 non-null     object
 1   content       16 non-null     object
 2   comment       16 non-null     int64 
 3   image_cnt     16 non-null     int64 
 4   video_cnt     16 non-null     int64 
 5   link_cnt      16 non-null     int64 
 6   hash_tag      1 non-null      object
 7   blog_tag      15 non-null     object
 8   categort_cnt  16 non-null     object
 9   date          16 non-null     object
dtypes: int64(4), object(6)
memory usage: 1.4+ KB
